<a href="https://colab.research.google.com/github/bhutamanav11/YardStick/blob/main/yardstick_internship.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!pip install --quiet --upgrade openai jsonschema


In [ ]:
!pip install --quiet --upgrade gitpython

In [32]:
# Securely get and set GROQ API key, create OpenAI-compatible client
import os
from getpass import getpass
from openai import OpenAI

# Try Colab userdata first (if you previously set it). Otherwise ask.
groq_key = None
try:
    from google.colab import userdata
    groq_key = userdata.get('GROQ_API_KEY')
except Exception:
    pass

if not groq_key:
    groq_key = getpass("Paste your GROQ API key (starts with gsk_): ")

# sanitize and set
groq_key = groq_key.strip()
os.environ["GROQ_API_KEY"] = groq_key

# create client
client = OpenAI(api_key=os.environ["GROQ_API_KEY"],
                base_url="https://api.groq.com/openai/v1")
print("Client created.")


Client created.


In [33]:
# List models and pick sensible defaults
try:
    resp = client.models.list()
    models = [m.id for m in resp.data]
    print("Available models (sample):", models[:30])
except Exception as e:
    print("Error listing models:", e)
    raise

# Choose defaults from the available list
SUMMARY_MODEL = "llama-3.3-70b-versatile" if "llama-3.3-70b-versatile" in models else (models[0] if models else None)
CLASSIFY_MODEL = "llama-3.1-8b-instant" if "llama-3.1-8b-instant" in models else (models[0] if models else None)

print("SUMMARY_MODEL:", SUMMARY_MODEL)
print("CLASSIFY_MODEL:", CLASSIFY_MODEL)


Available models (sample): ['openai/gpt-oss-20b', 'llama-3.3-70b-versatile', 'moonshotai/kimi-k2-instruct', 'meta-llama/llama-4-scout-17b-16e-instruct', 'meta-llama/llama-4-maverick-17b-128e-instruct', 'playai-tts', 'gemma2-9b-it', 'whisper-large-v3', 'moonshotai/kimi-k2-instruct-0905', 'llama-3.1-8b-instant', 'allam-2-7b', 'openai/gpt-oss-120b', 'groq/compound', 'deepseek-r1-distill-llama-70b', 'groq/compound-mini', 'whisper-large-v3-turbo', 'meta-llama/llama-prompt-guard-2-86m', 'playai-tts-arabic', 'qwen/qwen3-32b', 'meta-llama/llama-guard-4-12b', 'meta-llama/llama-prompt-guard-2-22m']
SUMMARY_MODEL: llama-3.3-70b-versatile
CLASSIFY_MODEL: llama-3.1-8b-instant


In [34]:
# Conversation manager: maintains history, truncation, k-th summarization
import json
from typing import List, Dict

class ConversationManager:
    def __init__(self, client, summary_model, k_summary=3):
        self.client = client
        self.summary_model = summary_model
        self.k_summary = max(1, int(k_summary))
        self.history: List[Dict] = []  # [{"role":..., "content":...}, ...]
        self.run_count = 0

    def add_message(self, role: str, content: str):
        """
        Add a message and trigger summarization every k_summary calls.
        If summarization runs, it REPLACES history with a single summary-system message.
        Returns the summary string when triggered, else None.
        """
        self.history.append({"role": role, "content": content})
        self.run_count += 1
        if self.run_count % self.k_summary == 0:
            summary = self.summarize_history()
            # store summary as single system message and reset run_count
            self.history = [{"role": "system", "content": f"[SUMMARY] {summary}"}]
            return summary
        return None

    def summarize_history(self, max_tokens=256, temperature=0.0):
        """
        Use LLM to summarize current conversation history.
        Output: brief 3-bullet summary string.
        """
        convo_text = "\n".join(f"{m['role']}: {m['content']}" for m in self.history)
        messages = [
            {"role": "system", "content": "You are a concise summarizer. Output up to 3 short bullet points."},
            {"role": "user", "content": f"Summarize this conversation:\n\n{convo_text}"}
        ]
        resp = self.client.chat.completions.create(
            model=self.summary_model,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens
        )
        return resp.choices[0].message.content.strip()

    def truncate_by_turns(self, last_n: int):
        """Keep only the last `last_n` messages (turns)."""
        if last_n <= 0:
            return
        self.history = self.history[-last_n:]

    def truncate_by_chars(self, max_chars: int):
        """Keep newest messages until total characters <= max_chars."""
        if max_chars <= 0:
            return
        out = []
        total = 0
        for msg in reversed(self.history):
            l = len(msg["content"])
            if total + l > max_chars:
                break
            out.insert(0, msg)
            total += l
        self.history = out

    def truncate_by_words(self, max_words: int):
        """Keep newest messages until total words <= max_words."""
        if max_words <= 0:
            return
        out = []
        total = 0
        for msg in reversed(self.history):
            w = len(msg["content"].split())
            if total + w > max_words:
                break
            out.insert(0, msg)
            total += w
        self.history = out

    def get_history(self):
        return self.history

    def save(self, filename="conversations.json"):
        with open(filename, "w") as f:
            json.dump(self.history, f, indent=2)


In [35]:
# Demo: show periodic summarization and truncation options
mgr = ConversationManager(client, SUMMARY_MODEL, k_summary=3)  # summarize every 3 add_message calls

# Multiple conversation samples
samples = [
    ("user", "Hi, I need help finishing my internship assignment."),
    ("assistant", "Sure — which part are you stuck on?"),
    ("user", "I need summarization and classification using Groq API."),
    ("assistant", "OK. Do you want periodic summarization?"),
    ("user", "Yes, summarize after every 3rd run. Also show truncation options."),
    ("assistant", "Got it. Please share sample chats."),
    ("user", "My name is Manav. Email: student@example.com. Phone: +91 9876543210. Location: Mumbai. Age: 21")
]

print("Feeding messages and showing when periodic summary triggers.\n")
for i, (role, text) in enumerate(samples, start=1):
    s = mgr.add_message(role, text)
    print(f"Added ({i}) {role}: {text}")
    if s:
        print(">>> PERIODIC SUMMARY TRIGGERED:\n", s)
    print("Current history:", json.dumps(mgr.get_history(), indent=2))
    print("-----")

# Show truncation examples
print("\n--- Demonstrate truncation by turns (keep last 3) ---")
mgr.truncate_by_turns(3)
print(json.dumps(mgr.get_history(), indent=2))

# Rebuild full history to show char/word truncation demo
mgr = ConversationManager(client, SUMMARY_MODEL, k_summary=100)  # avoid summarization for demo
for r, t in samples:
    mgr.add_message(r, t)

print("\n--- Truncate by characters (~80 chars) ---")
mgr.truncate_by_chars(80)
print(json.dumps(mgr.get_history(), indent=2))

# Rebuild again
mgr = ConversationManager(client, SUMMARY_MODEL, k_summary=100)
for r, t in samples:
    mgr.add_message(r, t)

print("\n--- Truncate by words (keep ~25 words) ---")
mgr.truncate_by_words(25)
print(json.dumps(mgr.get_history(), indent=2))

# Save final demo history
mgr.save("task1_conversations.json")
print("Saved task1_conversations.json")


Feeding messages and showing when periodic summary triggers.

Added (1) user: Hi, I need help finishing my internship assignment.
Current history: [
  {
    "role": "user",
    "content": "Hi, I need help finishing my internship assignment."
  }
]
-----
Added (2) assistant: Sure — which part are you stuck on?
Current history: [
  {
    "role": "user",
    "content": "Hi, I need help finishing my internship assignment."
  },
  {
    "role": "assistant",
    "content": "Sure \u2014 which part are you stuck on?"
  }
]
-----
Added (3) user: I need summarization and classification using Groq API.
>>> PERIODIC SUMMARY TRIGGERED:
 * You're working on an internship assignment and need help.
* The assignment involves summarization and classification tasks.
* You want to use the Groq API to accomplish these tasks.
Current history: [
  {
    "role": "system",
    "content": "[SUMMARY] * You're working on an internship assignment and need help.\n* The assignment involves summarization and classifi

In [36]:
# Task 2: Define schema and use function-calling to extract name,email,phone,location,age
import json, re
from jsonschema import validate, ValidationError

# 1) JSON Schema for validation
schema = {
  "type": "object",
  "properties": {
    "name": {"type": ["string","null"], "minLength": 0},
    "email": {"type": ["string","null"]},
    "phone": {"type": ["string","null"]},
    "location": {"type": ["string","null"]},
    "age": {"type": ["integer","null"], "minimum": 0, "maximum": 150}
  },
  "additionalProperties": False
}

# 2) OpenAI function definition (for function-calling)
function_def = {
    "name": "extract_user_info",
    "description": "Extract name, email, phone, location and age from a user message",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {"type": "string"},
            "email": {"type": "string"},
            "phone": {"type": "string"},
            "location": {"type": "string"},
            "age": {"type": "integer"}
        },
        "required": []
    }
}

# 3) Samples (>=3)
sample_chats = [
    "Hi, I'm Priya Sharma. My email is priya.sharma@example.com and I'm 24 years old, from Pune.",
    "Hello. Name: Manav Bhuta; phone +91 9876543210; location Mumbai. Age: 21. Contact: student@example.com",
    "This is Raj — raj.contact@gmail.com — I'm 30, living in Delhi. Phone: 09988776655"
]

def extract_function_args(resp):
    """
    Robustly extract JSON arguments from a function-calling response.
    """
    choice = resp.choices[0]
    # Try attribute access
    try:
        fc = choice.message.function_call
        args_str = fc.arguments
        return json.loads(args_str)
    except Exception:
        # Try dict-style fallback
        try:
            fc = choice["message"]["function_call"]
            args_str = fc["arguments"]
            return json.loads(args_str)
        except Exception:
            # fallback: try to find JSON in assistant content
            try:
                text = choice.message.content
            except Exception:
                text = str(choice)
            m = re.search(r'(\{.*\})', text, re.S)
            if m:
                return json.loads(m.group(1))
            raise RuntimeError("Unable to parse function call arguments from model response.")

parsed_results = []
for i, txt in enumerate(sample_chats, start=1):
    print(f"\nSample {i} input: {txt}")
    resp = client.chat.completions.create(
        model=CLASSIFY_MODEL,
        messages=[{"role":"system","content":"You are a strict JSON extractor. Use the provided function schema."},
                  {"role":"user","content":txt}],
        functions=[function_def],
        function_call={"name":"extract_user_info"},
        temperature=0.0,
        max_tokens=300
    )

    parsed = extract_function_args(resp)
    # Basic normalization: convert numeric age strings
    if parsed.get("age") is not None and isinstance(parsed.get("age"), str) and parsed.get("age").isdigit():
        parsed["age"] = int(parsed["age"])

    # attempt validation
    try:
        validate(instance=parsed, schema=schema)
        print("Parsed & VALID:", json.dumps(parsed, indent=2))
    except ValidationError as ve:
        print("Validation ERROR:", ve.message)
        print("Parsed (raw):", parsed)
    parsed_results.append(parsed)

# Save results
with open("task2_extractions.json","w") as f:
    json.dump(parsed_results, f, indent=2)
print("\nSaved task2_extractions.json")



Sample 1 input: Hi, I'm Priya Sharma. My email is priya.sharma@example.com and I'm 24 years old, from Pune.
Parsed & VALID: {
  "age": 24,
  "email": "priya.sharma@example.com",
  "location": "Pune",
  "name": "Priya Sharma",
  "phone": ""
}

Sample 2 input: Hello. Name: Manav Bhuta; phone +91 9876543210; location Mumbai. Age: 21. Contact: student@example.com
Parsed & VALID: {
  "age": 21,
  "email": "student@example.com",
  "location": "Mumbai",
  "name": "Manav Bhuta",
  "phone": "+91 9876543210"
}

Sample 3 input: This is Raj — raj.contact@gmail.com — I'm 30, living in Delhi. Phone: 09988776655
Parsed & VALID: {
  "age": 30,
  "email": "raj.contact@gmail.com",
  "location": "Delhi",
  "name": "Raj",
  "phone": "09988776655"
}

Saved task2_extractions.json


In [37]:
# Ensure generated files exist and list
!ls -lah

# Optionally download files to local machine
from google.colab import files
files.download("task1_conversations.json")
files.download("task2_extractions.json")


total 36K
drwxr-xr-x 1 root root 4.0K Sep 14 13:08 .
drwxr-xr-x 1 root root 4.0K Sep 14 12:13 ..
-rw-r--r-- 1 root root  981 Sep 14 12:40 classification.json
drwxr-xr-x 4 root root 4.0K Sep  9 13:46 .config
-rw-r--r-- 1 root root  422 Sep 14 12:40 conversations.json
drwxr-xr-x 1 root root 4.0K Sep  9 13:46 sample_data
-rw-r--r-- 1 root root  294 Sep 14 12:40 summary.txt
-rw-r--r-- 1 root root  229 Sep 14 13:08 task1_conversations.json
-rw-r--r-- 1 root root  411 Sep 14 13:08 task2_extractions.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>